In [ ]:

pd.set_option('display.max_colwidth', None)

# df = pd.read_excel("raw data/LG-A9K_REVIEWS.xlsx")

df = pd.read_csv("raw data/LG-A9K_REVIEWS.csv")
# df = pd.read_csv("raw data/LG-A9K_REVIEWS.csv", encoding='cp1252')
df.insert(0, 'ID', range(0 + len(df)))
df['REVIEW'] = df['TITLE'] + ". " + df['CONTENT']
df['REVIEW'] = df['REVIEW'].replace(np.nan, '')
df = df.drop(columns=['TITLE', 'CONTENT'])

print(df.head())
# Remove empty strings after colon and add space before Pros and Cons
df['REVIEW'] = df['REVIEW'].apply(
    lambda x: x.replace(":   ", ":").replace(":  ", ":").replace(": ", ":").replace(":\n\n", ": ").replace(":\n",
                                                                                                           ": ").replace(
        "\t", "").replace("\n-", "").replace("\n ", "\n"))
# df['REVIEW'][0]


df['REVIEW']


# Separate REVIEW into PARAGRAPHS
def separate_paragraphs(review):
    para_list = []
    paragraphs = review.split('\n\n')
    para_list.extend(paragraphs)
    return para_list


df['PARAGRAPHS'] = df['REVIEW'].apply(lambda x: separate_paragraphs(x))

# Remove empty strings from PARAGRAPHS
# df['PARAGRAPHS'] = df['PARAGRAPHS'].apply(lambda x: list(filter(None, x)))
# df['PARAGRAPHS'][0]

df['PARAGRAPHS'][0]
# Drop df columns: REVIEW
df = df.drop(columns=['RETAILER', 'PRODUCT', 'POST_DATE', 'REVIEWER_NAME', 'REVIEW'])

# Make PARAGRAPHS list to string
# df['PARAGRAPHS'] = df['PARAGRAPHS'].agg(lambda x: ','.join(map(str, x)))
df.head()

# df.insert(1, 'P_ID', range(0 + len(df)))


# explode the PARAGRAPHS Column
df = df.explode('PARAGRAPHS')
df = df.reset_index(drop=True)
# remove empty rows from PARAGRAPHS
df_paragraphs = df[df['PARAGRAPHS'] != '']

df_paragraphs

from transformers import AutoTokenizer, AutoModelForSequenceClassification

from scipy.special import softmax

MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
from tqdm.notebook import tqdm


def polarity_scores_roberta(review):
    encoded_text = tokenizer(review, padding=True, truncation=True, max_length=512, return_tensors='pt')
    encoded_text
    # print(encoded_text)
    output = model(**encoded_text)
    # output
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    scores
    scores_dict = {
        'negative': scores[0],
        'neutral': scores[1],
        'positive': scores[2]
    }
    return scores_dict


# polarity_scores_roberta(review)
df_paragraphs.insert(1, 'P_ID', range(0 + len(df_paragraphs)))

df_paragraphs

res = {}
for i, row in tqdm(df_paragraphs.iterrows(), total=len(df_paragraphs)):
    try:
        text = row['PARAGRAPHS']
        myid = row['P_ID']
        # vader_results = sia.polarity_scores(text)
        roberta_result = polarity_scores_roberta(text)
        res[myid] = {**roberta_result}
    except RuntimeError:
        print(f'Broke for id {myid}')

results_df = pd.DataFrame(res).T
results_df = results_df.reset_index().rename(columns={'index': 'P_ID'})
results_df = results_df.merge(df_paragraphs, how='left')
results_df.head()
# Keywords Extraction
# added below since giving a certificate verification error.
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('stopwords')
nltk.download('punkt')

# copy results_df to keywords_df
keywords_df = results_df.copy()
keywords_df['PARAGRAPHS'][524]
import string, re
from nltk import word_tokenize
from nltk.corpus import stopwords

for i in range(0, len(keywords_df['PARAGRAPHS'])):
    keywords_df['PARAGRAPHS'][i] = keywords_df['PARAGRAPHS'][i].translate(str.maketrans('', '', string.punctuation))
    keywords_df['PARAGRAPHS'][i] = keywords_df['PARAGRAPHS'][i].replace('\n', '. ')
    keywords_df['PARAGRAPHS'][i] = keywords_df['PARAGRAPHS'][i].lower()
    # keywords_df['PARAGRAPHS'][i] = re.sub("['\"]","",keywords_df['PARAGRAPHS'][i])
    for j in re.findall('"([^"]*)"', keywords_df['PARAGRAPHS'][i]):
        keywords_df['PARAGRAPHS'][i] = keywords_df['PARAGRAPHS'][i].replace('"{}"'.format(j), j.replace(' ', '_'))

keywords_df['PARAGRAPHS'][524]

# word_tokenize paragraphs

keywords_df['KEYWORD'] = keywords_df['PARAGRAPHS'].apply(lambda x: word_tokenize(x))
keywords_df['KEYWORD']
english_stopwords = stopwords.words('english')
for i in range(0, len(keywords_df['KEYWORD'])):
    keywords_df['KEYWORD'][i] = [w for w in keywords_df['KEYWORD'][i] if w.lower() not in english_stopwords]

# remove duplicate keywords_df['KEYWORD']
keywords_df['KEYWORD'] = keywords_df['KEYWORD'].apply(lambda x: list(dict.fromkeys(x)))
keywords_df['KEYWORD']
# apply rake_nltk_var.extract_keywords_from_text to each row in PARAGRAPHS
# keywords_df['KEYWORDS'] = keywords_df['PARAGRAPHS'].apply(lambda x: rake_nltk_var.extract_keywords_from_text(x))
# keywords_df['KEYWORDS'] = keywords_df['KEYWORDS'].apply(lambda x: rake_nltk_var.get_ranked_phrases())
# keywords_df
keywords_df['PARAGRAPHS'][524]
from rake_nltk import Metric

r = Rake(include_repeated_phrases=False,
         min_length=2,
         ranking_metric=Metric.WORD_DEGREE)
keywords = []
for i in range(0, len(keywords_df['PARAGRAPHS'])):
    keyword = r.extract_keywords_from_text(keywords_df['PARAGRAPHS'][i])
    keyword = r.get_ranked_phrases()
    keywords.append(keyword)
keywords_df['KEYWORDS'] = keywords
len(keywords)
keywords_df['KEYWORD'][524]
keywords_df['KEYWORDS'][524]
# save to csv
keywords_df.to_csv('keywords_df_RAKE.csv', encoding='utf-8-sig', index=False)

from pke.unsupervised import YAKE
from nltk.corpus import stopwords

document = "Machine learning (ML) is the study of computer algorithms that improve automatically through experience. It is seen as a subset of artificial intelligence."

# 1. Create YAKE keyword extractor
extractor = YAKE()

# 2. Load document
extractor.load_document(input=document,
                        language='en',
                        normalization=None)

# 3. Generate candidate 1-gram and 2-gram keywords
stoplist = stopwords.words('english')
extractor.candidate_selection(n=2, stoplist=stoplist)

# 4. Calculate scores for the candidate keywords
extractor.candidate_weighting(window=2,
                              stoplist=stoplist,
                              use_stems=False)

# 5. Select 10 highest ranked keywords
# Remove redundant keywords with similarity above 80%
key_phrases = extractor.get_n_best(n=10, threshold=0.8)
print(key_phrases)
import yake

document = "Machine learning (ML) is the study of computer algorithms that improve automatically through experience. It is seen as a subset of artificial intelligence."
kw_extractor = yake.KeywordExtractor()
keywords = kw_extractor.extract_keywords(document)

for kw in keywords:
    print(kw)

language = "en"
max_ngram_size = 3
deduplication_threshold = 0.9
deduplication_algo = 'seqm'
windowSize = 1
numOfKeywords = 20

custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold,
                                            dedupFunc=deduplication_algo, windowsSize=windowSize, top=numOfKeywords,
                                            features=None)
keywords = custom_kw_extractor.extract_keywords(text)

for kw in keywords:
    print(kw)